### Uso de Structured streaming con Spark ML.

Se recogen los datos en streaming desde Kafka, y se les aplicará el análisis de sentimiento para etiquetar el sentimiento de cada tweet recogido, utilizando Spark ML.

In [1]:
# imports y configuraciones necesarias
# Spark Streaming
from pyspark.streaming import StreamingContext  
# Kafka
from pyspark.streaming.kafka import KafkaUtils

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, NaiveBayes, RandomForestClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer, CountVectorizer, IDF
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit

from pyspark.mllib.util import *

from pyspark.ml.linalg import SparseVector, DenseVector

from pymongo import MongoClient

import pandas as pd

import json
import string
import re
import unicodedata

import time

import nltk
from nltk.corpus import stopwords
# cargamos las stopwords para cada idioma
spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')

# !pip install elasticsearch --si fuera necesario instalarlo
from elasticsearch import Elasticsearch
from elasticsearch import helpers


spark = SparkSession\
          .builder\
          .appName("twitter")\
          .master("spark://MacBook-Pro-de-Jose.local:7077")\
          .config("spark.io.compression.codec", "snappy")\
          .config("spark.streaming.stopGracefullyOnShutdown", True)\
          .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

#### Creamos las funciones necesarias.

In [2]:
# Función de carga de datos desde MongoDB de los datos almacenados desde Apache Nifi, tanto en inglés como español
def carga_datos_mongo():
    # conectamos con las tablas de Mongo desde donde cargamos los datos
    df_mongo_english = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
        .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_english").load()
    df_mongo_spanish = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
        .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_spanish").load()
    
    # nos quedamos solo con el atributo texto que es la información que nos interesa de cada dataset
    df_text_english = df_mongo_english[['text']]
    df_text_spanish = df_mongo_spanish[['text']]
    
    return df_text_english, df_text_spanish


# Función de carga de datos desde los csv generados con anterioridad con registros ya con el sentimiento anotado
def carga_datos_csv(csv):
    schema_csv = StructType([ 
        StructField("text", StringType(), True),
        StructField("sentiment", IntegerType(), True)
    ])
    
    # cargamos los csv
    if(csv=='english_full'): fichero = './data/df_result_english.csv'
    elif(csv=='spanish_full'): fichero = './data/df_result_spanish.csv'
    elif(csv=='english_neutro'): fichero = './data/df_result_english_neutral.csv'
    elif(csv=='spanish_neutro'): fichero = './data/df_result_spanish_neutral.csv'
    elif(csv=='english_noNeutro'): fichero = './data/df_result_english_noNeutral.csv'
    elif(csv=='spanish_noNeutro'): fichero = './data/df_result_spanish_noNeutral.csv'
    
    df_csv = sqlContext.\
        read.format("com.databricks.spark.csv").\
        option("header", "true").\
        option("inferschema", "true").\
        option("mode", "DROPMALFORMED").\
        schema(schema_csv).\
        load(fichero).\
        cache()
    
    return df_csv


# Funciones de visualización de DFs para ver las columnas, el número de registros o dimensiones, y el recuento
# de valores del atributo sentimiento en caso de tener la columna.
def visualizar_datos_csv(df):
    print("Columnas del dataframe: ", df.columns)
    print("Numero de registros = %d" % df.count())
    print("\n")
    print(df.limit(10).toPandas())
    print("\n")
    recuento_sentiment = df.select('sentiment').groupBy("sentiment").count().show()
    print("\n")

def visualizar_datos_mongo(df):
    df_pandas = df.toPandas()
    print("num_rows: %d\tColumnas: %d\n" % (df_pandas.shape[0], df_pandas.shape[1]) )
    print("Columnas:\n", list(df_pandas.columns))
    print(df_pandas.head(10))
    print("\n")


# función para eliminar palabras que no queramos analizar
def eliminar_stopwords(texto, palabras_eliminar):
    tok = nltk.tokenize
    palabras = tok.word_tokenize(texto)
        
    palabras_salida = []
        
    for palabra in palabras:
        if palabra not in palabras_eliminar:
            palabras_salida.append(palabra)
        
    salida = ""
    for i in range(len(palabras_salida)):
        if palabras_salida[i] in string.punctuation:
            salida = salida.strip()+palabras_salida[i] + " "
        else:
            salida += palabras_salida[i] + " "

    return salida


# Funciones de limpieza de los tweets
def limpieza_tweets_spanish(tokens : list) -> list:
    tweet = [re.sub('  +', ' ', s).strip() for s in tokens]
    tweet = [re.sub(r'http\S+', '', s) for s in tweet]  
    tweet = [re.sub(r'@[\S]+', '', s) for s in tweet]
    tweet = [re.sub(r'#(\S+)', r' \1 ', s) for s in tweet]
    tweet = [re.sub(r'\brt\b', '', s) for s in tweet]
    tweet = [re.sub(r'\.{2,}', ' ', s) for s in tweet]
    tweet = [re.sub(r'\s+', ' ', s) for s in tweet]
    tweet = [re.sub('','',s).lower() for s in tweet]
    # eliminar full_text que es como comienzan los textos que son extendidos
    tweet = [re.sub(r'full_text', '', s) for s in tweet] 
    
    # convertir la repetición de una letra más de 2 veces a 1
    # biennnnn --> bien
    tweet = [re.sub(r'(.)\1+', r'\1\1', s) for s in tweet]
    # remover - & '
    tweet = [re.sub(r'(-|\')', '', s) for s in tweet] 
    # eliminar acentos
    tweet = [''.join((c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c) != 'Mn')) for s in tweet]
        
    # reemplazar emojis
    emoji_pattern = re.compile(u'['u'\U0001F300-\U0001F64F'u'\U0001F680-\U0001F6FF'u'\
                               \u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
 
    tweet = [emoji_pattern.sub(r' ', s) for s in tweet]
    tweet = [re.sub("[^A-Za-z]+$",'',s) for s in tweet]
    tweet = [re.sub("^[^A-Za-z]+",'',s) for s in tweet]
    tweet = [re.sub("[\$*&!?///\º\'\’\‘\|()%/\"{}@;:+\[\]\–\”\…\“\】\【=]",'',s) for s in tweet]  
    
    # remover stopwords
    tweet = [eliminar_stopwords(s, spanish_stopwords) for s in tweet]

    filtered = filter(None, tweet)
    
    return list(filtered)

# a los datos en inglés le aplicamos sus stopwords correspondientes y no les quitamos los acentos
def limpieza_tweets_english(tokens : list) -> list:
    tweet = [re.sub('  +', ' ', s).strip() for s in tokens]
    tweet = [re.sub(r'http\S+', '', s) for s in tweet]  
    tweet = [re.sub(r'@[\S]+', '', s) for s in tweet]
    tweet = [re.sub(r'#(\S+)', r' \1 ', s) for s in tweet]
    tweet = [re.sub(r'\brt\b', '', s) for s in tweet]
    tweet = [re.sub(r'\.{2,}', ' ', s) for s in tweet]
    tweet = [re.sub(r'\s+', ' ', s) for s in tweet]
    tweet = [re.sub('','',s).lower() for s in tweet]
    # eliminar full_text que es como comienzan los textos que son extendidos
    tweet = [re.sub(r'full_text', '', s) for s in tweet] 
    
    # convertir la repetición de una letra más de 2 veces a 1
    # biennnnn --> bien
    tweet = [re.sub(r'(.)\1+', r'\1\1', s) for s in tweet]
    # remover - & '
    tweet = [re.sub(r'(-|\')', '', s) for s in tweet] 

    # reemplazar emojis
    emoji_pattern = re.compile(u'['u'\U0001F300-\U0001F64F'u'\U0001F680-\U0001F6FF'u'\
                               \u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
 
    tweet = [emoji_pattern.sub(r' ', s) for s in tweet]
    tweet = [re.sub("[^A-Za-z]+$",'',s) for s in tweet]
    tweet = [re.sub("^[^A-Za-z]+",'',s) for s in tweet]
    tweet = [re.sub("[\$*&!?///\º\'\’\‘\|()%/\"{}@;:+\[\]\–\”\…\“\】\【=]",'',s) for s in tweet]  
    
    # remover stopwords
    tweet = [eliminar_stopwords(s, english_stopwords) for s in tweet]

    filtered = filter(None, tweet)
    
    return list(filtered)


# Función de preprocesado de los datos
def preprocesado_dataframe(df1, df2, tipo):
    # primero usamos tokenizer y vamos a partir los tweets por palabras
    if(tipo==0): tokenizer = Tokenizer(inputCol = "text", outputCol = "token")
    elif(tipo==1): tokenizer = Tokenizer(inputCol = "texto", outputCol = "token")

    df_tokens_english = tokenizer.transform(df1)
    df_tokens_spanish = tokenizer.transform(df2)  
    
    # usamos las funciones de limpieza y preprocesado con ambos DFs ya con los textos divididos en tokens
    limpiezaUDF_english = F.udf(limpieza_tweets_english, ArrayType(StringType()))
    limpiezaUDF_spanish = F.udf(limpieza_tweets_spanish, ArrayType(StringType()))

    df_tokens_english = df_tokens_english.withColumn("tokens_clean", limpiezaUDF_english(df_tokens_english["token"]))
    df_tokens_spanish = df_tokens_spanish.withColumn("tokens_clean", limpiezaUDF_spanish(df_tokens_spanish["token"]))

    df_tokens_english = df_tokens_english.drop("token")
    df_tokens_spanish = df_tokens_spanish.drop("token")

    df_tokens_english_clean = df_tokens_english.where(F.size(F.col("tokens_clean")) > 0)
    df_tokens_spanish_clean = df_tokens_spanish.where(F.size(F.col("tokens_clean")) > 0)
    
    return df_tokens_english_clean, df_tokens_spanish_clean


# Función evaluación de modelo
def evaluar_modelo(metric, predCol, labelCol, pred1, pred2):
    evaluator = MulticlassClassificationEvaluator(metricName=metric, predictionCol=predCol, labelCol=labelCol)

    eval_eng_clean = evaluator.evaluate(pred1)
    print("-------------------------------------------------------")
    print("Medidas de rendimiento datos en inglés")
    print("-------------------------------------------------------")
    print("F1-score = %f" % eval_eng_clean)
    print("\n")
    
    eval_spa_clean = evaluator.evaluate(pred2)
    print("-------------------------------------------------------")
    print("Medidas de rendimiento datos en español")
    print("-------------------------------------------------------")
    print("F1-score = %f" % eval_spa_clean)
    print("\n") 


# Función para ver los mejores parámetros de un modelo al usar cross-validation
def mejores_parametros(model1, model2):
    bestPipeline_eng = model1.bestModel
    bestPipeline_spa = model2.bestModel

    bestVectorizer_eng = bestPipeline_eng.stages[0]
    bestVectorizer_spa = bestPipeline_spa.stages[0]

    bestParamsVect_eng = bestVectorizer_eng.extractParamMap()
    print ("Vectorizer parameters datos inglés:")
    for k in bestParamsVect_eng.keys():
        print ("  ", k, bestParamsVect_eng[k])
    
    bestParamsVect_spa = bestVectorizer_spa.extractParamMap()
    print("\n") 
    print ("Vectorizer parameters datos español:")
    for k in bestParamsVect_spa.keys():
        print ("  ", k, bestParamsVect_spa[k])

    bestModel_eng = bestPipeline_eng.stages[2]
    bestModel_spa = bestPipeline_spa.stages[2]

    bestParamsModel_eng = bestModel_eng.extractParamMap()
    print("\n") 
    print("Model parameters datos inglés:")
    for k in bestParamsModel_eng.keys():
        print("  ", k, bestParamsModel_eng[k])

    bestParamsModel_spa = bestModel_spa.extractParamMap()
    print("\n") 
    print("Model parameters datos español:")
    for k in bestParamsModel_spa.keys():
        print("  ", k, bestParamsModel_spa[k]) 

#### Carga de los datos guardados en formato .csv para entrenar el modelo a usar con los tweets que vienen en streaming.

In [3]:
# cargamos los distintos csv generados con anterioridad y con el sentimiento ya etiquetado
df_csv_english_full = carga_datos_csv('english_full')
df_csv_spanish_full = carga_datos_csv('spanish_full')

In [4]:
# cogemos un sample del DF con datos en inglés ya que tiene muchos datos y da problemas de cómputo y tiempos
df_csv_english_sample = df_csv_english_full.sample(False, 0.1, 45)

visualizar_datos_csv(df_csv_english_sample)

Columnas del dataframe:  ['text', 'sentiment']
Numero de registros = 175818


                                                text  sentiment
0  @VirginAmerica amazing to me that we can't get...          0
1  @VirginAmerica View of downtown Los Angeles, t...          2
2  @VirginAmerica plz help me win my bid upgrade ...          1
3  @VirginAmerica I'm #elevategold for a good rea...          2
4  @VirginAmerica @ladygaga @carrieunderwood Juli...          1
5  @VirginAmerica I’m having trouble adding this ...          0
6  @VirginAmerica you have the absolute best team...          2
7  @VirginAmerica has flight number 276 from SFO ...          1
8  @VirginAmerica Another delayed flight? #liking...          0
9  @VirginAmerica Can you find us a flt out of LA...          1


+---------+-----+
|sentiment|count|
+---------+-----+
|        1| 1372|
|        2|87372|
|        0|87074|
+---------+-----+





In [5]:
visualizar_datos_csv(df_csv_spanish_full)

Columnas del dataframe:  ['text', 'sentiment']
Numero de registros = 46787


                                                text  sentiment
0  @PauladeLasHeras No te libraras de ayudar me/n...          1
1                          @marodriguezb Gracias MAR          2
2  Off pensando en el regalito Sinde, la que se v...          0
3  Conozco a alguien q es adicto al drama! Ja ja ...          2
4  Toca @crackoviadeTV3 . Grabación dl especial N...          2
5  Buen día todos! Lo primero mandar un abrazo gr...          2
6  Desde el escaño. Todo listo para empezar #endi...          2
7  Bdías. EM no se ira de puente. Si vosotros os ...          2
8  Un sistema económico q recorta dinero para pre...          2
9                  #programascambiados caca d ajuste          0


+---------+-----+
|sentiment|count|
+---------+-----+
|        1| 2927|
|        2|25392|
|        0|18468|
+---------+-----+





#### Generamos el modelo que se probó anteriormente y dio los mejores resultados, y lo entrenamos con los datos recogidos de los csv.

In [6]:
# llamamos a la función que engloba el preprocesado de los datos, con la tokenización y la limpieza de tweets
df_tokens_english_clean, df_tokens_spanish_clean = preprocesado_dataframe(df_csv_english_sample,df_csv_spanish_full,0)

In [7]:
df_tokens_english_clean.limit(5).toPandas()

,text,sentiment,tokens_clean
0,@VirginAmerica amazing to me that we can't get...,0,"[amazing , cant , get , cold , air , vents , v..."
1,"@VirginAmerica View of downtown Los Angeles, t...",2,"[view , downtown , los , angeles , hollywood ,..."
2,@VirginAmerica plz help me win my bid upgrade ...,1,"[plz , help , win , bid , upgrade , flight , l..."
3,@VirginAmerica I'm #elevategold for a good rea...,2,"[im , elevategold , good , reason , rock ]"
4,@VirginAmerica @ladygaga @carrieunderwood Juli...,1,"[julie , andrews , first , lady , gaga , wowd ..."


In [8]:
df_tokens_spanish_clean.limit(5).toPandas()

,text,sentiment,tokens_clean
0,@PauladeLasHeras No te libraras de ayudar me/n...,1,"[libraras , ayudar , menos , besos , gracias ]"
1,@marodriguezb Gracias MAR,2,"[gracias , mar ]"
2,"Off pensando en el regalito Sinde, la que se v...",0,"[off , pensando , regalito , sinde , va , sgae..."
3,Conozco a alguien q es adicto al drama! Ja ja ...,2,"[conozco , alguien , q , adicto , drama , ja ,..."
4,Toca @crackoviadeTV3 . Grabación dl especial N...,2,"[toca , grabacion , dl , especial , navideno m..."


In [9]:
# a partir del dataframe anterior, vamos a eliminar la columna texto en los datos que han pasado por la limpieza y
# preprocesado.
df_tokens_english_clean = df_tokens_english_clean.drop("text")
df_tokens_spanish_clean = df_tokens_spanish_clean.drop("text")

In [10]:
# cambiamos el nombre de la columna sentiment por label a los dataframes
df_english_clean = df_tokens_english_clean.withColumnRenamed("sentiment", "label").cache()
df_spanish_clean = df_tokens_spanish_clean.withColumnRenamed("sentiment", "label").cache()

In [11]:
# generación de conjunto de training y test de los dataframes
train_english_clean, test_english_clean = df_english_clean.randomSplit([0.9, 0.1], seed=12345)
train_spanish_clean, test_spanish_clean = df_spanish_clean.randomSplit([0.9, 0.1], seed=12345)

In [12]:
# generamos el pipeline del modelo con sus etapas y parámetros, además de utilizar TrainValidationSplit
# sería la configuración del modelo con el que se obtuvieron los mejores resultados en el notebook de pruebas de ML
countVec = CountVectorizer(inputCol="tokens_clean", outputCol="countfeatures")
idf = IDF(inputCol=countVec.getOutputCol(), outputCol="features", minDocFreq=2)
lr = LogisticRegression(maxIter=10, regParam=0.001, elasticNetParam=0)

pipeline = Pipeline(stages=[countVec, idf, lr])

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01, 0.005]) \
    .addGrid(lr.elasticNetParam, [0.1, 0.2])\
    .addGrid(lr.maxIter, [5, 20])\
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(countVec.minTF, [1.0, 3.0, 5.0])\
    .build()

tvs = TrainValidationSplit(estimator=pipeline,
                           estimatorParamMaps=paramGrid,
                           evaluator=MulticlassClassificationEvaluator(),
                           trainRatio=0.9)

In [33]:
# entrenamos los modelos
model_eng = tvs.fit(train_english_clean)

In [34]:
model_spa = tvs.fit(train_spanish_clean)

In [35]:
# usamos los modelos para predecir usando los datos de test
pred_eng = model_eng.transform(test_english_clean)
pred_spa = model_spa.transform(test_spanish_clean)

In [36]:
# mostrar los mejores parámetros
mejores_parametros(model_eng, model_spa)

Vectorizer parameters datos inglés:
   CountVectorizer_684b6f2e96fc__binary False
   CountVectorizer_684b6f2e96fc__maxDF 9.223372036854776e+18
   CountVectorizer_684b6f2e96fc__minDF 1.0
   CountVectorizer_684b6f2e96fc__minTF 1.0
   CountVectorizer_684b6f2e96fc__outputCol countfeatures
   CountVectorizer_684b6f2e96fc__vocabSize 262144
   CountVectorizer_684b6f2e96fc__inputCol tokens_clean


Vectorizer parameters datos español:
   CountVectorizer_684b6f2e96fc__binary False
   CountVectorizer_684b6f2e96fc__maxDF 9.223372036854776e+18
   CountVectorizer_684b6f2e96fc__minDF 1.0
   CountVectorizer_684b6f2e96fc__minTF 1.0
   CountVectorizer_684b6f2e96fc__outputCol countfeatures
   CountVectorizer_684b6f2e96fc__vocabSize 262144
   CountVectorizer_684b6f2e96fc__inputCol tokens_clean


Model parameters datos inglés:
   LogisticRegression_0e605d2eb517__aggregationDepth 2
   LogisticRegression_0e605d2eb517__elasticNetParam 0.1
   LogisticRegression_0e605d2eb517__family auto
   LogisticRegression_0

In [38]:
# evaluamos el modelo
evaluar_modelo('f1', 'prediction', 'label', pred_eng, pred_spa)

-------------------------------------------------------
Medidas de rendimiento datos en inglés
-------------------------------------------------------
F1-score = 0.761035


-------------------------------------------------------
Medidas de rendimiento datos en español
-------------------------------------------------------
F1-score = 0.801400




#### Conectamos con el streaming para recoger los datos de Kafka y tenerlos en un dataframe.

In [13]:
# definimos el schema de los datos que queremos guardar de los datos de entrada al sistema
schema = StructType([
    StructField("id", LongType(), True),
    StructField("text", StringType(), True),
    StructField("truncated", BooleanType(), True),
    StructField("extended_tweet", StringType(), True),
    StructField("favorite_count", IntegerType(), True),
    StructField("quote_count", IntegerType(), True),
    StructField("reply_count", IntegerType(), True),
    StructField("retweet_count", IntegerType(), True),
    StructField("sentiment", IntegerType(), True)
])

In [14]:
# se obtienen desde Kafka los datos del topic en español mediante schema
# cambiamos a latest startingOffsets, y True en failOnDataLoss. 
df_spanish = spark\
.readStream\
.format("kafka")\
.option("kafka.bootstrap.servers", "localhost:9092")\
.option("startingOffsets", "latest")\
.option("subscribe", "TopicSpanish")\
.option("failOnDataLoss", "true") \
.load()

df_spanish = df_spanish.selectExpr("CAST(value AS STRING)")

df_spanish = df_spanish.select(F.from_json(F.col("value"), schema).alias("data")).select("data.*")           
                                                                                          
df_spanish.createOrReplaceTempView("spanish_schema")

In [15]:
display(df_spanish)

DataFrame[id: bigint, text: string, truncated: boolean, extended_tweet: string, favorite_count: int, quote_count: int, reply_count: int, retweet_count: int, sentiment: int]

In [16]:
# se obtienen desde Kafka los datos del topic en inglés mediante schema
df_english = spark\
.readStream\
.format("kafka")\
.option("kafka.bootstrap.servers", "localhost:9093")\
.option("startingOffsets", "latest")\
.option("subscribe", "TopicEnglish")\
.option("failOnDataLoss", "true") \
.load()

df_english = df_english.selectExpr("CAST(value AS STRING)")

df_english = df_english.select(F.from_json(F.col("value"), schema).alias("data")).select("data.*")

df_english.createOrReplaceTempView("english_schema")

In [17]:
display(df_english)

DataFrame[id: bigint, text: string, truncated: boolean, extended_tweet: string, favorite_count: int, quote_count: int, reply_count: int, retweet_count: int, sentiment: int]

In [18]:
# chequeamos los datos en streaming
print(" ")
print("Is the stream ready?")
print(df_spanish.isStreaming, df_english.isStreaming)

 
Is the stream ready?
True True


In [19]:
# sumamos las 4 columnas del recuento de replies, favorites, quotes y retweet para tener una sola columna
df_spanish = df_spanish.withColumn('interacciones', df_spanish.favorite_count\
                + df_spanish.quote_count + df_spanish.reply_count + df_spanish.retweet_count)
df_english = df_english.withColumn('interacciones', df_english.favorite_count\
                + df_english.quote_count + df_english.reply_count + df_english.retweet_count)

# eliminamos las columnas sumadas para el total anterior
columns_to_drop = ['favorite_count', 'quote_count', 'reply_count', 'retweet_count']
df_spanish = df_spanish.drop(*columns_to_drop)
df_english = df_english.drop(*columns_to_drop)

In [20]:
# vamos ahora a ver el valor de truncated, si es True, el texto del tweet está truncado en la columna text y por tanto
# debemos coger el texto de la columna extended_tweet. Para ello usamos una función udf.
def udf_cambiar_texto(col1,col2,col3):
    if(col1==True):
        col2=col3
    return col2

cambiar_texto_udf=F.udf(udf_cambiar_texto)

In [21]:
# aplicamos la función udf a los dataframes. Generamos una columna 'texto' que tendrá el texto final, ya sea el 
# inicial de la columna 'text' o si está truncado será el que venga en la columna 'extended_tweet'
df_spanish = df_spanish.select('*').withColumn("texto", cambiar_texto_udf("truncated","text","extended_tweet"))
df_english = df_english.select('*').withColumn("texto", cambiar_texto_udf("truncated","text","extended_tweet"))

In [22]:
# eliminar columnas y dejar los dataframes como queremos finalmente ordenando las columnas si es necesario
columns = ['id', 'texto', 'interacciones', 'sentiment']
df_spanish = df_spanish[columns] 
df_english = df_english[columns]

In [23]:
display(df_spanish)
display(df_english)

DataFrame[id: bigint, texto: string, interacciones: int, sentiment: int]

DataFrame[id: bigint, texto: string, interacciones: int, sentiment: int]

In [24]:
# definimos querys en memoria para ver los datos que se han recogido
query_spanish = df_spanish.writeStream.outputMode("append").queryName("spanish").format("memory")\
.option("truncate", "False").start()

query_english = df_english.writeStream.outputMode("append").queryName("english").format("memory")\
.option("truncate", "False").start()

print(query_spanish.status)
print(query_english.status)

{'message': 'Getting offsets from KafkaV2[Subscribe[TopicSpanish]]', 'isDataAvailable': False, 'isTriggerActive': True}
{'message': 'Getting offsets from KafkaV2[Subscribe[TopicEnglish]]', 'isDataAvailable': False, 'isTriggerActive': True}


In [25]:
result_spanish = spark.table("spanish")
result_english = spark.table("english")

In [26]:
print("Número de registros en español: ", result_spanish.count())
print("Número de registros en inglés: ", result_english.count())

Número de registros en español:  24
Número de registros en inglés:  76


In [27]:
# visualizamos los datos en español
result_spanish.limit(20).toPandas()

,id,texto,interacciones,sentiment
0,1167074214811242497,RT @FiltracionesB: Ahora si!! #benvindoneymar !,0,None
1,1167074214827966464,RT amirnorman #MaiaReficco #KallysMashupShow #...,0,None
2,1167074214815383555,RT @DaniCubidesF: Afortunados ustedes que no h...,0,None
3,1167074214832168960,"RT @FutbolFrancais: Según Sky Italia, el ficha...",0,None
4,1167074214819577857,@Ushld_loveme Valid 😭😭,0,None
5,1167074214844743682,"@CamiiAguilarrr Jajaja, no se 🤷‍♀️🤷‍♀️",0,None
6,1167074214806982656,El valiente. https://t.co/0LIosZ4yoZ,0,None
7,1167074214815289344,@CHOMPS Y estoooooo? 😂,0,None
8,1167074214836416512,"{""full_text"":""@Malote39 @Marconav1981 @Jupol_Z...",0,None
9,1167074214819454976,RT @xxvane99xx: 100 RT y subo más 💦 https://t....,0,None


In [28]:
# visualizamos los datos en inglés
result_english.limit(20).toPandas()

,id,texto,interacciones,sentiment
0,1167074210642092032,RT @FBlankenshipWSB: This!! https://t.co/yVFRN...,0,None
1,1167074210646085632,SARKAR real verdict enna ?\nBLOCKBUSTER OR SUP...,0,None
2,1167074210637844480,"{""full_text"":""@america_katz Nero (the left) fi...",0,None
3,1167074210650263552,"{""full_text"":""or recycling sentences on the In...",0,None
4,1167074210637762560,"{""full_text"":""Chelsea, Tottenham, Liverpool an...",0,None
5,1167074210642104320,@shakguerrero awantiaaaaaaaaaaaaaaaaa https://...,0,None
6,1167074214807031808,RT @SocDogFacts: when you think your dog is ba...,0,None
7,1167074214823809024,you gotta tell me the reason why we can't fall...,0,None
8,1167074214840602624,@KallMe_Kris @BreeziNUrMouth If he didn’t do i...,0,None
9,1167074214815227904,RT @russellsidhu2: Turning cigarette buds into...,0,None


In [29]:
# end streaming
query_spanish.stop()
query_english.stop()

#### Ahora aplicamos a los DFs recogidos en streaming el modelo entrenado para etiquetar el sentimiento de cada tweet.

Vamos también a probar a limpiar estos datos, y aplicamos el modelo para predecir el sentimiento.

In [30]:
# llamamos a la función que engloba el preprocesado de los datos, con la tokenización y la limpieza de tweets
df_tokens_english, df_tokens_spanish = preprocesado_dataframe(result_english,result_spanish,1)

In [31]:
df_english_clean = df_tokens_english.withColumnRenamed("sentiment", "label").cache()
df_spanish_clean = df_tokens_spanish.withColumnRenamed("sentiment", "label").cache()

In [39]:
# usamos el modelo para predecir el sentimiento de los tweets en los distintos DFs por idioma
pred_eng = model_eng.transform(df_english_clean)
pred_spa = model_spa.transform(df_spanish_clean)

In [40]:
pred_eng = pred_eng.drop("label","countfeatures","features","rawPrediction","tokens_clean")
pred_spa = pred_spa.drop("label","countfeatures","features","rawPrediction","tokens_clean")

In [41]:
# función udf para calcular fecha y hora
def add_timestamp():
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    return timestamp

add_timestamp_udf = F.udf(add_timestamp, StringType())

In [42]:
# añadimos la columna con el timestamp a los datos
pred_eng = pred_eng.withColumn("timestamp", add_timestamp_udf())
pred_spa = pred_spa.withColumn("timestamp", add_timestamp_udf())

In [43]:
# la columna de probabilidad se genera de tipo DenseVector, vamos a crear una función para pasarla a Array y poder
# tratarla mejor.
def sparse_to_array(v):
    v = DenseVector(v)
    new_array = list([float(x) for x in v])
    return new_array

sparse_to_array_udf = F.udf(sparse_to_array, ArrayType(FloatType()))

pred_eng = pred_eng.withColumn('probability_array', sparse_to_array_udf('probability'))
pred_spa = pred_spa.withColumn('probability_array', sparse_to_array_udf('probability'))

In [44]:
# generamos una función udf para según el valor del sentimiento calculado por el modelo, nos quedamos con el valor
# del array de probabilidad que corresponde a esa etiqueta de sentimiento, que será el mayor.
def udf_probability(col1,col2):
    if(col2==0.0):
        col1=col1[0]
    elif(col2==1.0):
        col1=col1[1]
    elif(col2==2.0):
        col1=col1[2]
        
    return col1

probability_udf=F.udf(udf_probability)

pred_eng = pred_eng.withColumn('probabilidad', probability_udf('probability_array','prediction'))
pred_spa = pred_spa.withColumn('probabilidad', probability_udf('probability_array','prediction'))

In [45]:
# ordenamos las columnas en el orden que queremos
columns = ['id', 'texto', 'interacciones', 'probabilidad', 'prediction', 'timestamp']
pred_eng = pred_eng[columns] 
pred_spa = pred_spa[columns]

Nos quedamos en los dataframes con las siguientes columnas:
- ID: id único del tweet
- Texto: el texto del tweet
- Interacciones: sería la suma de respuestas, citas, retweets y favoritos del tweet.
- Probability: probabilidad con la que se obtiene el valor del sentimiento de cada tweet.
- Prediction: etiqueta de sentimiento elegida por el modelo para el tweet.
- Timestamp: añadimos fecha y hora del procesamiento.

In [46]:
print("Predictions:")
pred_eng.limit(20).toPandas()

Predictions:


,id,texto,interacciones,probabilidad,prediction,timestamp
0,1167074210646085632,SARKAR real verdict enna ?\nBLOCKBUSTER OR SUP...,0,0.6209715008735657,2.0,2019-08-29 16:16:00
1,1167074210637844480,"{""full_text"":""@america_katz Nero (the left) fi...",0,0.7965912818908691,2.0,2019-08-29 16:16:00
2,1167074210650263552,"{""full_text"":""or recycling sentences on the In...",0,0.9550488591194153,0.0,2019-08-29 16:16:00
3,1167074210637762560,"{""full_text"":""Chelsea, Tottenham, Liverpool an...",0,0.5845345854759216,0.0,2019-08-29 16:16:00
4,1167074210642104320,@shakguerrero awantiaaaaaaaaaaaaaaaaa https://...,0,0.575066089630127,2.0,2019-08-29 16:16:00
5,1167074214807031808,RT @SocDogFacts: when you think your dog is ba...,0,0.7224486470222473,0.0,2019-08-29 16:16:00
6,1167074214823809024,you gotta tell me the reason why we can't fall...,0,0.5488168001174927,0.0,2019-08-29 16:16:00
7,1167074214840602624,@KallMe_Kris @BreeziNUrMouth If he didn’t do i...,0,0.8246603012084961,0.0,2019-08-29 16:16:00
8,1167074214815227904,RT @russellsidhu2: Turning cigarette buds into...,0,0.7484982013702393,2.0,2019-08-29 16:16:02
9,1167074214806880262,RT @superjanella: btw.... HAHA GOTCHU ALL 👅😈 \...,0,0.9120923280715942,2.0,2019-08-29 16:16:02


In [47]:
print("Predictions:")
pred_spa.limit(20).toPandas()

Predictions:


,id,texto,interacciones,probabilidad,prediction,timestamp
0,1167074214811242497,RT @FiltracionesB: Ahora si!! #benvindoneymar !,0,0.38416776061058044,0.0,2019-08-29 16:16:08
1,1167074214827966464,RT amirnorman #MaiaReficco #KallysMashupShow #...,0,0.3333333432674408,0.0,2019-08-29 16:16:08
2,1167074214815383555,RT @DaniCubidesF: Afortunados ustedes que no h...,0,0.6482715010643005,2.0,2019-08-29 16:16:08
3,1167074214832168960,"RT @FutbolFrancais: Según Sky Italia, el ficha...",0,0.9297542572021484,0.0,2019-08-29 16:16:09
4,1167074214819577857,@Ushld_loveme Valid 😭😭,0,0.3333333432674408,0.0,2019-08-29 16:16:09
5,1167074214844743682,"@CamiiAguilarrr Jajaja, no se 🤷‍♀️🤷‍♀️",0,0.7352712154388428,2.0,2019-08-29 16:16:09
6,1167074214806982656,El valiente. https://t.co/0LIosZ4yoZ,0,0.5833292007446289,2.0,2019-08-29 16:16:09
7,1167074214815289344,@CHOMPS Y estoooooo? 😂,0,0.3333333432674408,0.0,2019-08-29 16:16:09
8,1167074214836416512,"{""full_text"":""@Malote39 @Marconav1981 @Jupol_Z...",0,0.7096914052963257,0.0,2019-08-29 16:16:09
9,1167074214819454976,RT @xxvane99xx: 100 RT y subo más 💦 https://t....,0,0.5462927222251892,2.0,2019-08-29 16:16:08


#### Almacenar datos en CSV, MongoDB y ElasticSearch.

Probamos a almacenar los datos etiquetados en diferentes formatos: archivos csv, MongoDB, y en ElasticSearch para su posterior visualización con Kibana.

In [48]:
# sacamos una cadena con fecha y hora para añadirla al nombre de los csv que se generan a continuación
timestr = time.strftime("%Y%m%d-%H%M%S")
print(timestr)

20190829-161613


In [49]:
# pasamos los dataframes con el sentimiento calculado a pandas
df_eng_pred = pred_eng.toPandas()
df_spa_pred = pred_spa.toPandas()

In [50]:
# se guardan los dataframes con el sentimiento calculado en csv
df_eng_pred.to_csv('./data/predict_streaming_english_'+timestr+'.csv', index=False)
df_spa_pred.to_csv('./data/predict_streaming_spanish_'+timestr+'.csv', index=False)

In [51]:
# guardar datos en MongoDB
conexion = 'mongodb://localhost:27017'
client = MongoClient(conexion)

# accedemos a la base de datos
db = client.tfm_twitter
# insertamos los dataframes en la tabla correspondiente
db.tweets_streaming_english.insert_many(df_eng_pred.to_dict("records"))
db.tweets_streaming_spanish.insert_many(df_spa_pred.to_dict("records"))

In [52]:
# guardar datos en ElasticSearch
es = Elasticsearch('http://localhost:9200/')

# añadimos fecha y hora a los índices
indice_spa = "tweets_sentiment_spa_"+timestr
indice_eng = "tweets_sentiment_eng_"+timestr

# primero comprobamos si ya existen los índices y se borran
if es.indices.exists(indice_spa):
    !curl -X DELETE localhost:9200/{indice_spa}
if es.indices.exists(indice_eng):
    !curl -X DELETE localhost:9200/{indice_eng}

# generación de índices
!curl -X PUT localhost:9200/{indice_spa}
!curl -X PUT localhost:9200/{indice_eng}

TYPE = "record"

def rec_to_actions(df, lang):
    for record in df.to_dict(orient="records"):
        if(lang==0): yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (indice_spa, TYPE))
        elif(lang==1): yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (indice_eng, TYPE))
        yield (json.dumps(record, default=int))

if not es.indices.exists(indice_spa):
    raise RuntimeError('index does not exists, use `curl -X PUT "localhost:9200/%s"` and try again'%indice_spa)
if not es.indices.exists(indice_eng):
    raise RuntimeError('index does not exists, use `curl -X PUT "localhost:9200/%s"` and try again'%indice_eng)

r_spa = es.bulk(rec_to_actions(df_spa_pred, 0))
r_eng = es.bulk(rec_to_actions(df_eng_pred, 1)) 

print(not r_spa["errors"], not r_eng["errors"])

{"acknowledged":true,"shards_acknowledged":true,"index":"tweets_sentiment_spa_20190829-161613"}{"acknowledged":true,"shards_acknowledged":true,"index":"tweets_sentiment_eng_20190829-161613"}True True


In [53]:
# se guardan los modelos utilizados para la predicción
model_eng_best = model_eng.bestModel
model_spa_best = model_spa.bestModel
model_eng_best.write().overwrite().save('./data/models/model_eng_'+timestr)
model_spa_best.write().overwrite().save('./data/models/model_spa_'+timestr)

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
